# Intro

## Libraries

In [1]:
%matplotlib inline
import pandas as pd, datetime as dt
#from co2mpas.datasync import _get_interp_method
import collections
import numpy as np
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import glob, os
import time

In [2]:
speed_bin = 10

In [3]:
# bins
a = np.arange(10,140,speed_bin)
a

array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130])

In [4]:
indices = [3,4,5,6,7,8,9]
np.take(a, indices)


array([ 40,  50,  60,  70,  80,  90, 100])

In [5]:
# labels
np.arange(5*speed_bin/10,140,speed_bin)

array([   5.,   15.,   25.,   35.,   45.,   55.,   65.,   75.,   85.,
         95.,  105.,  115.,  125.,  135.])

## Input files

In [2]:
car = {'Audi A1': ['./Audi A1/', 'xls',98,'gasoline'],             # we could get vela2 PN data from the analysis_veh.xls DB
      'BMW 530d': ['./BMW 530d/', 'xlsx',124,'diesel'],
      'Audi A3': ['./Audi A3/', 'xls',120,'diesel'],               # specify the FWD for NEDC in function lab_a1 
      'Citroen Cactus': ['./Citroen Cactus/','xls',95,'diesel'],   # specify WLTP_COLD in function lab_a1
      'Fiat 500x': ['./Fiat 500x/','xls',144,'diesel'],
      'Opel Astra': ['./Opel Astra/','',103,'gasoline'],           # Alternative code because of Vela8 file format (.xls for WLTP and .xlsx for NEDC)
      'Fiat Panda': ['./Fiat Panda/', 'xlsx',119,'gasoline'],
      'Kia Sportage': ['./Kia Sportage/','xlsx',124,'diesel'],
      'Peugeot Partner': ['./Peugeot Partner/','xls',112,'diesel'],
      'Porsche Cayenne': ['./Porsche Cayenne/','xlsx',179,'diesel'],
      'Renault Twingo': ['./Renault Twingo/', 'xlsx',112,'gasoline'],
      'VW Golf': ['./VW Golf/','xlsx',117,'diesel']
      }

In [3]:
vehicle = 'Audi A1'

In [4]:
path = car[vehicle][0]
filetype = car[vehicle][1]
t_a = car[vehicle][2]
fuel = car[vehicle][3]

## Create the dataframe

In [ ]:
"""
Only to be used the first time
"""

"""
###############################
Only to be used the first time!
###############################
"""

arrays = [['fuel','T_A',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE'],
           ['-', '-', 
           'CO2','CO2','CO2','CO2','CO2',             # reorder columns
           'NOx','NOx','NOx','NOx','NOx',
           'CO','CO','CO','CO','CO',
           'HC','HC','HC','HC','HC',
           'PN','PN','PN','PN','PN',
           'CO2','CO2','CO2','CO2','CO2','CO2','CO2',  
           'NOx','NOx','NOx','NOx','NOx','NOx','NOx',
           'CO','CO','CO','CO','CO','CO','CO',
           'HC','HC','HC','HC','HC','HC','HC',
           'PN','PN','PN','PN','PN','PN','PN',
           'CO2','CO2','CO2','CO2','CO2',  
           'NOx','NOx','NOx','NOx','NOx',
           'CO','CO','CO','CO','CO',
           'HC','HC','HC','HC','HC',
           'PN','PN','PN','PN','PN'],
           ['-', '-',
           'complete','urban','extraurban','150/d','150',
           'complete','urban','extraurban','150/d','150',
           'complete','urban','extraurban','150/d','150',
           'complete','urban','extraurban','150/d','150',
           'complete','urban','extraurban','150/d','150',
           'complete','low','medium','high','extrahigh','150/d','150',
           'complete','low','medium','high','extrahigh','150/d','150',
           'complete','low','medium','high','extrahigh','150/d','150',
           'complete','low','medium','high','extrahigh','150/d','150',
           'complete','low','medium','high','extrahigh','150/d','150',
           'complete','urban','rural','motorway','150',
           'complete','urban','rural','motorway','150',
           'complete','urban','rural','motorway','150',
           'complete','urban','rural','motorway','150',
           'complete','urban','rural','motorway','150']
         ]

tuples = list(zip(*arrays))

my_columns = pd.MultiIndex.from_tuples(tuples, names=['cycle', 'pollutant','phase'])

my_index = []

RegPollEmi = pd.DataFrame(index=my_index, columns=my_columns)
#RegPollEmi.columns